# dnd programming lol

In [1]:
# importing necessary modules
import random
import numpy as np
import pandas as pd

In [2]:
# calculing minimum, maximum and average initiative over 10k iterations
def individual_initiative(name, init_bonus, roll_type='single'):
    # initializing the variables
    total_initiative = 0
    iterations = 10000

    # initializing the 10k iterations, rolling initiative for each
    for _ in range(iterations + 1):
        if roll_type == 'adv':
            roll_result = max([random.randint(1, 20),
                               random.randint(1, 20)])
        else:
            roll_result = random.randint(1, 20)
        total_initiative += roll_result + init_bonus

    min_initiative = 1 + init_bonus # rolling a 1 + initiative bonus
    max_initiative = 20 + init_bonus # rolling a 20 + initiative bonus
    avg_initiative = total_initiative / iterations # average over 10k rolls
    print(f"{name}'s average initiative is {avg_initiative:.0f}.")
    print(f"{name}'s minimum initiative is {min_initiative:.0f}.")
    print(f"{name}'s maximum initiative is {max_initiative:.0f}.")

In [3]:
# sample character data to calculate initiative
individual_initiative(name= 'Nana', init_bonus= 4,
                      roll_type= 'adv')

Nana's average initiative is 18.
Nana's minimum initiative is 5.
Nana's maximum initiative is 24.


In [4]:
# # calculing minimum, maximum and average team initiative over 10k iterations
def team_initiative(team_data):
    initiatives = []
    
    for name, details in team_data.items():
        # initializing extracting variables from team data dictionary
        init_bonus = details['initiative_bonus']
        roll_type = details['roll_type']

        total_initiative = 0
        iterations = 10000
    
        for _ in range(iterations + 1):
            if roll_type == 'adv':
                roll_result = max([random.randint(1, 20),
                                   random.randint(1, 20)])
            else:
                roll_result = random.randint(1, 20)
            total_initiative += roll_result + init_bonus

        min_initiative = 1 + init_bonus
        max_initiative = 20 + init_bonus
        avg_initiative = total_initiative / iterations

        # appending individual initiatives to constitue the team
        initiatives.append({
        'character_name': name,
        'min_initiative': min_initiative,
        'avg_initiative': np.round(avg_initiative, 1),
        'max_initiative': max_initiative
        })
    
    team_initiatives = pd.DataFrame(initiatives)
    return team_initiatives

In [5]:
# sample team data to calculate initiative
party_initiatives = {
    'Nana': {'initiative_bonus': 4, 'roll_type': 'adv'},
    'Mana': {'initiative_bonus': 8, 'roll_type': 'single'},
    'Paris': {'initiative_bonus': 2, 'roll_type': 'single'}
}

team_initiative(party_initiatives)

,character_name,min_initiative,avg_initiative,max_initiative
0,Nana,5,17.8,24
1,Mana,9,18.5,28
2,Paris,3,12.4,22


In [6]:
# calculating probability of success of each skill per each DC
def skill_success(character_data): 
    # initializing lists of DCs and to hold results
    results = []
    dc_values = [5, 10, 15, 20, 25, 30]
    
    for skill, details in character_data.items():
        skill_mod = details['skill_modifier']
        roll_type = details['roll_type']

        # instancing calculations for each type of roll 
        for dc in dc_values:
            def single_roll(save_mod, dc):
                probability = max(0.01, min((21 - (dc - save_mod)) / 20, 1))
                if probability < 0.05:
                    return 0.05
                elif probability > 0.95:
                    return 0.95
                return probability
    
            def advantage(save_mod, dc):
                single_prob = single_roll(save_mod, dc)
                probability = 1 - (1 - single_prob) ** 2
                if probability < 0.1:
                    return 0.1
                elif probability > 0.9975:
                    return 0.9975
                return probability
    
            def disadvantage(save_mod, dc):
                single_prob = single_roll(save_mod, dc)
                probability = single_prob ** 2
                if probability < 0.025:
                    return 0.025
                elif probability > 0.9025:
                    return 0.9025
                return probability
        
            if roll_type == 'adv':
                hit_chance = advantage(skill_mod, dc)
            elif roll_type == 'dis':
                hit_chance = disadvantage(skill_mod, dc)
            else:
                hit_chance = single_roll(skill_mod, dc)

            success_rate = hit_chance * 100

            # appending the result of each DC for each skill
            results.append({
            'skill': skill,
            'dc': dc,
            'success%': f"{round(success_rate, 2)}%"
            })

    # returning a dataframe
    prob_df = pd.DataFrame(results)
    skill_df = prob_df.pivot(index='skill',
                             columns= 'dc',
                             values= ['success%'])
    return skill_df

In [7]:
# sample character data to calculate skill probabilities
Nana_skills = {
    'Acrobatics': {'skill_modifier': 6, 'roll_type': 'single'},
    'Animal Handling': {'skill_modifier': 5, 'roll_type': 'single'},
    'Arcana': {'skill_modifier': 0, 'roll_type': 'single'},
    'Athletics': {'skill_modifier': -1, 'roll_type': 'adv'},
    'Deception': {'skill_modifier': 2, 'roll_type': 'single'},
    'History': {'skill_modifier': 0, 'roll_type': 'single'},
    'Insight': {'skill_modifier': 9, 'roll_type': 'single'},
    'Intimidation': {'skill_modifier': 2, 'roll_type': 'adv'},
    'Investigation': {'skill_modifier': 0, 'roll_type': 'single'},
    'Medicine': {'skill_modifier': 5, 'roll_type': 'single'},
    'Nature': {'skill_modifier': 4, 'roll_type': 'single'},
    'Perception': {'skill_modifier': 9, 'roll_type': 'single'},
    'Performance': {'skill_modifier': 6, 'roll_type': 'single'},
    'Persuasion': {'skill_modifier': 6, 'roll_type': 'single'},
    'Religion': {'skill_modifier': 0, 'roll_type': 'single'},
    'Sleight of Hand': {'skill_modifier': 2, 'roll_type': 'single'},
    'Stealth': {'skill_modifier': 2, 'roll_type': 'single'},
    'Survival': {'skill_modifier': 5, 'roll_type': 'single'}
}

skill_success(Nana_skills)

success%                                      
dc                    5       10      15      20     25     30
skill                                                         
Acrobatics         95.0%   85.0%   60.0%   35.0%  10.0%   5.0%
Animal Handling    95.0%   80.0%   55.0%   30.0%   5.0%   5.0%
Arcana             80.0%   55.0%   30.0%    5.0%   5.0%   5.0%
Athletics         93.75%   75.0%  43.75%   10.0%  10.0%  10.0%
Deception          90.0%   65.0%   40.0%   15.0%   5.0%   5.0%
History            80.0%   55.0%   30.0%    5.0%   5.0%   5.0%
Insight            95.0%   95.0%   75.0%   50.0%  25.0%   5.0%
Intimidation       99.0%  87.75%   64.0%  27.75%  10.0%  10.0%
Investigation      80.0%   55.0%   30.0%    5.0%   5.0%   5.0%
Medicine           95.0%   80.0%   55.0%   30.0%   5.0%   5.0%
Nature             95.0%   75.0%   50.0%   25.0%   5.0%   5.0%
Perception         95.0%   95.0%   75.0%   50.0%  25.0%   5.0%
Performance        95.0%   85.0%   60.0%   35.0%  10.0%   5.0%
Persuasion         95.0%   85.0%   60.0%   35.0%  10.0%   5.0%
Religion           80.0%   55.0%   30.0%    5.0%   5.0%   5.0%
Sleight of Hand    90.0%   65.0%   40.0%   15.0%   5.0%   5.0%
Stealth            90.0%   65.0%   40.0%   15.0%   5.0%   5.0%
Survival           95.0%   80.0%   55.0%   30.0%   5.0%   5.0%

In [8]:
# calculating probability of success of each save per each DC
def save_success(character_data): 
    # initializing lists of DCs and to hold results
    results = []
    dc_values = [5, 10, 15, 20, 25, 30]
    
    for save, details in character_data.items():
        save_mod = details['save_modifier']
        roll_type = details['roll_type']

        # instancing calculations for each type of roll
        for dc in dc_values:
            def single_roll(save_mod, dc):
                probability = max(0.01, min((21 - (dc - save_mod)) / 20, 1))
                if probability < 0.05:
                    return 0.05
                elif probability > 0.95:
                    return 0.95
                return probability
                
    
            def advantage(save_mod, dc):
                single_prob = single_roll(save_mod, dc)
                probability = 1 - (1 - single_prob) ** 2
                if probability < 0.1:
                    return 0.1
                elif probability > 0.9975:
                    return 0.9975
                return probability
    
            def disadvantage(save_mod, dc):
                single_prob = single_roll(save_mod, dc)
                probability = single_prob ** 2
                if probability < 0.025:
                    return 0.025
                elif probability > 0.9025:
                    return 0.9025
                return probability
        
            if roll_type == 'adv':
                hit_chance = advantage(save_mod, dc)
            elif roll_type == 'dis':
                hit_chance = disadvantage(save_mod, dc)
            else:
                hit_chance = single_roll(save_mod, dc)

            success_rate = hit_chance * 100

            # appending the result of each DC for each skill
            results.append({
            'save': save,
            'dc': dc,
            'success%': f"{round(success_rate, 2)}%"
            })

    # returning a dataframe
    prob_df = pd.DataFrame(results)
    save_df = prob_df.pivot(index='save',
                             columns= 'dc',
                             values= ['success%'])
    return save_df

In [9]:
# sample character data to calculate save probabilities
Nana_saves = {
    'Strength': {'save_modifier': 8, 'roll_type': 'single'},
    'Dexterity': {'save_modifier': 4, 'roll_type': 'single'},
    'Constitution': {'save_modifier': 6, 'roll_type': 'single'},
    'Intelligence': {'save_modifier': -1, 'roll_type': 'single'},
    'Wisdom': {'save_modifier': 2, 'roll_type': 'single'},
    'Charisma': {'save_modifier': 0, 'roll_type': 'single'}
}

save_success(Nana_saves)

success%                                  
dc                 5      10     15     20     25    30
save                                                   
Charisma        80.0%  55.0%  30.0%   5.0%   5.0%  5.0%
Constitution    95.0%  85.0%  60.0%  35.0%  10.0%  5.0%
Dexterity       95.0%  75.0%  50.0%  25.0%   5.0%  5.0%
Intelligence    75.0%  50.0%  25.0%   5.0%   5.0%  5.0%
Strength        95.0%  95.0%  70.0%  45.0%  20.0%  5.0%
Wisdom          90.0%  65.0%  40.0%  15.0%   5.0%  5.0%

In [10]:
# calculating team stealthing ability with and without pass without a trace
def team_stealth(team_data):
    stealths = []

    # instancing the variables and iterations based on roll type
    for name, details in team_data.items():
        stealth_mod = details['stealth_mod']
        roll_type = details['roll_type']

        total_stealth = 0
        total_buff_stealth = 0
        iterations = 10000
    
        for _ in range(iterations + 1):
            if roll_type == 'adv':
                roll_result = max([random.randint(1, 20),
                                   random.randint(1, 20)])
            elif roll_type == 'dis':
                roll_result = min([random.randint(1, 20),
                                   random.randint(1, 20)])
            else:
                roll_result = random.randint(1, 20)
            total_stealth += roll_result + stealth_mod
            total_buff_stealth += roll_result + stealth_mod + 10

        # min, max and average stealth with and without pass without a trace
        min_stealth = 1 + stealth_mod
        max_stealth = 20 + stealth_mod
        pass_wo_trace_min = 1 + stealth_mod + 10
        pass_wo_trace_max = 20 + stealth_mod + 10
        avg_stealth = total_stealth / iterations
        pass_wo_trace_avg = total_buff_stealth / iterations
        
        # returning a dictionary with everyone's stealth
        stealths.append({
        'name': name,
        'min_stealth': min_stealth,
        'Pass_wo_trace_min': pass_wo_trace_min,
        'avg_stealth': np.round(avg_stealth, 1),
        'Pass_wo_trace_avg': np.round(pass_wo_trace_avg, 1),
        'max_stealth': max_stealth,
        'Pass_wo_trace_max': pass_wo_trace_max
        })
    
    return pd.DataFrame(stealths)

In [11]:
# sample team data to calculate stealth
party_stealths = {
    'Nana': {'stealth_mod': 2, 'roll_type': 'single'},
    'Mana': {'stealth_mod': 12, 'roll_type': 'single'},
    'Paris': {'stealth_mod': 6, 'roll_type': 'adv'}
}

stealth_df = team_stealth(party_stealths)
stealth_df

,name,min_stealth,Pass_wo_trace_min,avg_stealth,Pass_wo_trace_avg,max_stealth,Pass_wo_trace_max
0,Nana,3,13,12.6,22.6,22,32
1,Mana,13,23,22.5,32.5,32,42
2,Paris,7,17,19.9,29.9,26,36


In [12]:
# parsing string like '1d6+5' into dice damage and bonus to damage
def parse_damage(damage):
    # parsing the bonus from the dice damage
    if '+' in damage:
        dice, bonus = damage.split('+')
        bonus_dmg = int(bonus)
    else:
        dice = damage
        bonus_dmg = 0

    # split damage to get # dice & # sides per die
    dice_count, dice_sides = map(int, dice.split('d'))

    return dice_count, dice_sides, bonus_dmg

# taking in # of attacks, 2 different types of damaage with overrides if relevant
def total_dmg_round(name, max_attacks, 
                    damage_1, override_1=None, 
                    damage_2=None, override_2=None):
    # split bonus_dmg from dice_dmg
    dice_count_1, dice_sides_1, bonus_dmg_1 = parse_damage(damage_1)

    # min, avg, max dmg calculations per attack for the 1st type of damage
    if override_1:
        min_dmg_per_attack_1 = bonus_dmg_1 + dice_count_1 * override_1
    else:
        min_dmg_per_attack_1 = bonus_dmg_1 + dice_count_1 * 1
    max_dmg_per_attack_1 = bonus_dmg_1 + dice_count_1 * dice_sides_1
    avg_dmg_per_attack_1 = (bonus_dmg_1 + dice_count_1 * (dice_sides_1 + 1)) / 2

    total_min_dmg = min_dmg_per_attack_1
    total_max_dmg = max_dmg_per_attack_1
    total_avg_dmg = avg_dmg_per_attack_1

    # min, avg, max dmg calculations per attack for the 2nd type of damage
    if damage_2:
        dice_count_2, dice_sides_2, bonus_dmg_2 = parse_damage(damage_2) 

        # min, avg, max dmg calculs per attack
        if override_2:
            min_dmg_per_attack_2 = bonus_dmg_2 + dice_count_2 * override_2
        else:
            min_dmg_per_attack_2 = bonus_dmg_2 + dice_count_2 * 1
        max_dmg_per_attack_2 = bonus_dmg_2 + dice_count_2 * dice_sides_2
        avg_dmg_per_attack_2 = (bonus_dmg_2 + dice_count_2 * (dice_sides_2 + 1)) / 2

        total_min_dmg += min_dmg_per_attack_2
        total_max_dmg += max_dmg_per_attack_2
        total_avg_dmg += avg_dmg_per_attack_2
    
    damage_results = []

    # calculating cumulative damage for each attack that hits
    for hits in range(1, max_attacks + 1):
        cumul_min_dmg = total_min_dmg * hits
        cumul_max_dmg = total_max_dmg * hits
        cumul_avg_dmg = total_avg_dmg * hits

        damage_results.append((hits, cumul_min_dmg, 
                               cumul_avg_dmg, cumul_max_dmg))
    
    # returning the character's name and all results
    print(f"Total damage per round of {name}")
    return pd.DataFrame(damage_results,
                       columns= ['hit#', 'min_total',\
                                 'avg_total', 'max_total'])

In [13]:
# parsing damage from the druid spell Wrath of Nature from Paris
wrath_nature = total_dmg_round(name= 'Paris', max_attacks= 1,
                               damage_1= '4d6', damage_2= '3d8')
wrath_nature

Total damage per round of Paris


,hit#,min_total,avg_total,max_total
0,1,7,27.5,48


In [14]:
# parsing damage from our fighter Nana with 3 attacks per turn
Nana = total_dmg_round(name= 'Nana', max_attacks= 3,
                               damage_1= '1d12+5', damage_2= '2d6')
Nana

Total damage per round of Nana


,hit#,min_total,avg_total,max_total
0,1,8,16.0,29
1,2,16,32.0,58
2,3,24,48.0,87
